In [ ]:
# # Distance Per Dollar
# # You’re given a dataset of uber rides with the traveling distance (‘distance_to_travel’) and cost (‘monetary_cost’) for each ride.
# # For each date, find the difference between the distance-per-dollar for that date and the average distance-per-dollar for that year-month.
# # Distance-per-dollar is defined as the distance traveled divided by the cost of the ride.
# # The output should include the year-month (YYYY-MM) and the average difference in distance-per-dollar for said year-month as an absolute value rounded to the 2nd decimal. 
# # You should also count both success and failed request_status as the distance and cost values are populated for all ride requests. 
# # Also, assume that all dates are unique in the dataset.
# # Order your results by earliest request date first.






import pandas as pd
import numpy as np
from datetime import datetime
uber_request_logs=pd.read_csv('uber_request_logs.csv')
 
uber_request_logs['month_year'] = uber_request_logs['request_date'].apply(pd.to_datetime)

uber_request_logs['month_year'] = pd.to_datetime(uber_request_logs['request_date']).dt.to_period('M')
uber_request_logs['monthly_dist_to_cost'] = uber_request_logs['distance_to_travel'] / \
    uber_request_logs['monetary_cost']
uber_request_logs["avg_monthly_dist_cost"] = uber_request_logs.groupby(['month_year'])['distance_to_travel'].transform(
    'sum') / uber_request_logs.groupby(['month_year'])['monetary_cost'].transform('sum')
uber_request_logs['mean_deviation'] = round(np.abs(
    uber_request_logs['monthly_dist_to_cost'] - uber_request_logs["avg_monthly_dist_cost"]), 2)

result = uber_request_logs.groupby(['month_year'])['mean_deviation'].mean(
).reset_index()
result['mean_deviation'] = round(result['mean_deviation'],2)
result



# with a as (select request_date, (distance_to_travel/monetary_cost)dc,TO_CHAR(request_date, 'YYYY-MM') mnth,
# avg(distance_to_travel/monetary_cost)over(partition by TO_CHAR(request_date, 'YYYY-MM')) 
# , 
# round(abs((distance_to_travel/monetary_cost)-avg(distance_to_travel/monetary_cost)over(partition by TO_CHAR(request_date, 'YYYY-MM'))) ::decimal,2) as diff
#   from uber_request_logs
# order by 1 desc)
# select mnth request_mnth ,avg(diff) difference from a
# group by 1
# order by 1 

In [ ]:
# Import your libraries
import pandas as pd
from datetime import datetime

uber_request_logs=pd.read_csv('uber_request_logs.csv')
uber_request_logs['request_date']=pd.to_datetime(uber_request_logs['request_date'], errors='coerce')
uber_request_logs['month_year'] = uber_request_logs['request_date'].dt.to_period('M') 
# print(uber_request_logs)  
# uber_request_logs['month_year'] = uber_request_logs.request_date.dt.strftime('%Y-%m')
# print(uber_request_logs) 
uber_request_logs['distance_per_dollar'] = uber_request_logs['distance_to_travel']  / uber_request_logs['monetary_cost']

avg_amts = uber_request_logs.groupby('month_year')['distance_per_dollar'].mean().reset_index()
avg_amts.columns = ['month_year', 'avg_distance_per_dollar']

uber_request_logs = uber_request_logs.merge(avg_amts, on='month_year')

uber_request_logs['difference_between'] = abs(uber_request_logs['distance_per_dollar'] - uber_request_logs['avg_distance_per_dollar']).round(2)

uber_request_logs.groupby('month_year')['difference_between'].mean().reset_index()

In [ ]:
# Import your libraries
import pandas as pd
import numpy as np
import datetime 
# Start writing code
uber_request_logs=pd.read_csv('uber_request_logs.csv')

uber_request_logs.head()
df = uber_request_logs

df['request_date'] = pd.to_datetime(df['request_date'])
df['date'] = df['request_date'].dt.to_period('M')
df['distance_dollar'] = df['distance_to_travel'] / df['monetary_cost']

df['average_distance'] = df.groupby('date')['distance_to_travel'].transform('sum')

df['monetary_cost'] = df.groupby('date')['monetary_cost'].transform('sum')
df['average_distance_dollar'] = df['average_distance']/ df['monetary_cost'] 



df['diff'] = round(np.abs(df['distance_dollar'] - df['average_distance_dollar']),2)
print('df')
print(df)
result = df.groupby('date')['diff'].mean().to_frame('avr').reset_index()

result['avr'] = round(result['avr'],2)
result

In [ ]:
# Find the total costs and total customers acquired in each year
# Find the total costs and total customers acquired in each year.
# Output the year along with corresponding total money spent and total acquired customers.

import pandas as pd
import numpy as np
uber_advertising=pd.read_csv('uber_advertising.csv')

result = uber_advertising.groupby(['year']).agg({'money_spent':'sum','customers_acquired':'sum'}).reset_index()
result

# # select year,sum(customers_acquired) total_customers_acquired,sum(money_spent) total_money_spent
# # from uber_advertising

# # group by 1

In [ ]:

a=uber_advertising.groupby('year')['money_spent'].sum().reset_index()
b=uber_advertising.groupby('year')['customers_acquired'].sum().reset_index()
df=a.merge(b,on='year')
df

In [5]:
# Positive Ad Channels
# Find the advertising channel with the smallest maximum yearly spending that still brings in more than 1500 customers each year.
import pandas as pd
import numpy as np
uber_advertising=pd.read_csv('uber_advertising.csv')

grouped = uber_advertising.groupby(['advertising_channel']).agg({'customers_acquired':'min','money_spent':'max'}).reset_index()
print(grouped)
result = grouped[(grouped['customers_acquired']>1500)].sort_values('money_spent') 
result


# with a as (select  advertising_channel,max(money_spent) mx
# from uber_advertising
# where customers_acquired>1500
# group by 1
# order by 2)
# select  advertising_channel,mx
# from a
# where mx=(select min(mx) from a)
   

  advertising_channel  customers_acquired  money_spent
0          billboards                1800       200200
1               buses                2300        70000
2            busstops                 400         7000
3         celebrities                1800       123555
4               radio                  19         1500
5                  tv                4700       300000


,advertising_channel,customers_acquired,money_spent
1,buses,2300,70000
3,celebrities,1800,123555
0,billboards,1800,200200
5,tv,4700,300000


In [10]:
# Naive Forecasting
# Some forecasting methods are extremely simple and surprisingly effective.
# Naïve forecast is one of them; we simply set all forecasts to be the value of the last observation. 
# Our goal is to develop a naïve forecast for a new metric called "distance per dollar" defined as the (distance_to_travel/monetary_cost) in our dataset and measure its accuracy.

# To develop this forecast, aggregate 'distance per dollar' values at a monthly level. 
# This value becomes your actual value for the current month.
# The next step is to populate the forecasted value for each month. 
# This can be achieved simply by getting the previous month's value in a separate column.
# Now, we have actual and forecasted values.
# This is your naïve forecast.
# Let’s evaluate our model by calculating an error matrix called root mean squared error (RMSE).
# RMSE is defined as sqrt(mean(square(actual - forecast)). 
# Report out the RMSE rounded to the 2nd decimal spot.




# Import your libraries
import pandas as pd
import numpy as np
uber_request_logs=pd.read_csv('uber_request_logs.csv')

x = uber_request_logs
x['request_date'] = pd.to_datetime(x['request_date'])
# df['date'] = df['request_date'].dt.to_period('M')
x.request_date = x.request_date.dt.strftime('%Y-%m')
# print(x)
x = x.groupby('request_date', as_index=False).sum()
# print(x)
x['distance_per_dollar'] = x.distance_to_travel/x.monetary_cost
# print(x)
x['forecast'] = x['distance_per_dollar'].shift()
print(x)
rmse = np.round(np.sqrt(np.mean((x['distance_per_dollar']-x['forecast'])**2)),2)
rmse

# with a as (select extract(month from request_date) as month,
# sum(distance_to_travel) sd ,sum(monetary_cost) sc   
# from uber_request_logs 
# group by 1
# order by 1
# ),
# b as (
# select month,sd,sc ,
# LAG(sd,1) OVER ( ORDER BY month ) previous_sd,
# LAG(sc,1) OVER ( ORDER BY month ) previous_sc
# from a),
# c as (
# select month ,sd/sc actual ,previous_sd/previous_sc forecast

# from b),
# d as (select  month, POWER(actual-forecast,2) p  from c),
# e as 
# (select sqrt(avg(p))avg_rental from d)
# select ROUND(avg_rental :: decimal,2 )  from e

  request_date  request_id  distance_to_travel  monetary_cost  \
0      2020-01           2               93.36          22.68   
1      2020-02           4               61.58           8.04   
2      2020-03           6               45.57           4.68   
3      2020-04           8               48.22          11.20   
4      2020-05          46              140.34          51.89   
5      2020-06          27               54.30          25.03   
6      2020-07          38              162.47          38.00   
7      2020-08          35              178.81          29.96   
8      2020-09          24               98.77          38.55   
9      2020-10          20               53.76          22.94   

   driver_to_client_distance  distance_per_dollar  forecast  
0                      19.90             4.116402       NaN  
1                      44.26             7.659204  4.116402  
2                      24.19             9.737179  7.659204  
3                      48.82        

2.75

In [14]:
# Highest Total Miles
# You’re given a table of Uber rides that contain the mileage and the purpose for the business expense.  
# You’re asked to find business purposes that generate the most miles driven for passengers that use Uber for their business transportation. 
# Find the top 3 business purpose categories by total mileage.


# Import your libraries
import pandas as pd
my_uber_drives=pd.read_csv('my_uber_drives.csv')

# Start writing code

x = my_uber_drives.groupby('purpose')["miles"].sum().to_frame("total_miles_per_business").reset_index().sort_values('total_miles_per_business', ascending=False)[:3]
x

# select   purpose,sum(miles)
# from my_uber_drives
# where purpose is not null
# group by 1
# order by 2 desc
# limit 3 

,purpose,total_miles_per_business
7,Meeting,2851.3
4,Customer Visit,2089.5
6,Meal/Entertain,911.7
